### 1. plot location ###

In [8]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime
import random
from tqdm import tqdm

def read_nldas(file_dir, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):
        nldas_file = 'NLDAS_'+str(yr)+'.nc'
        nldas_path = os.path.join(file_dir, nldas_file)
        f_nldas = xr.open_dataset(nldas_path)
        
        if yr == start_yr:
            pcp_concat = f_nldas['pcp'].values[:] # (time, y, x). unit: mm/day
            tmean_concat = f_nldas['t_mean'].values[:] # (time, y, x). unit: degC
#             t_range_concat = f_nldas['t_range'].values[:]
            time_concat = f_nldas['time'].values[:]
        else:
            pcp_concat = np.concatenate((pcp_concat, f_nldas['pcp'].values[:]), axis = 0)
            tmean_concat = np.concatenate((tmean_concat, f_nldas['t_mean'].values[:]), axis = 0)
#             trange_concat = np.concatenate((trange_concat, f_nldas['t_range'].values[:]), axis = 0)
            time_concat = np.concatenate((time_concat, f_nldas['time'].values[:]), axis = 0)

    # get time mask from nldas data
    time_obj = pd.to_datetime(time_concat)    
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 

    # convert masked values to nan
    time_obj = time_obj[mask_t]
    pcp_concat=np.where(mask_xy==0,np.nan,pcp_concat[mask_t,:,:])
    tmean_concat=np.where(mask_xy==0,np.nan,tmean_concat[mask_t,:,:])

    return time_obj, pcp_concat, tmean_concat#, t_range_concat

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
test_folder =test_folders[-1]

regress_subforlder = 'gmet_regr'
ens_subforlder = 'gmet_ens_bc'

time_format = '%Y-%m-%d'

dpi_value = 150
date_starts = ['2016-02-01','2016-08-01'] # list of start dates of plot time periods
date_ends = ['2016-02-29','2016-08-31']   # list of end dates of plot time periods
grids = [[40,265],[41,264]]               # xid and yid of the spatial distribution
# date_starts = ['2016-08-09']
# date_ends = ['2016-08-20']

random.seed(10)

output_dir=os.path.join(root_dir, 'scripts/step35_statistical_perturb_NLDAS')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
#======================================================================================================
# Read gridinfo mask
print('Read gridinfo mask')
# get xy mask from gridinfo.nc
f_gridinfo = xr.open_dataset(gridinfo_file)
mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
latitude = f_gridinfo['latitude'].values[:]
longitude = f_gridinfo['longitude'].values[:]

# prepare for basemap plot
llcrnrlon = longitude[0,0]
urcrnrlon = longitude[-1,-1]
llcrnrlat = latitude[0,0]
urcrnrlat = latitude[-1,-1]
lat_0=0.5*(llcrnrlat+urcrnrlat)
lon_0=0.5*(llcrnrlon+urcrnrlon)
(ny,nx)=np.shape(longitude)

#======================================================================================================
print('plot')
nrow = len(date_starts) # Wet, dry events
ncol = 2 # NLDAS, GMET std, statistical std    
fig = plt.figure(figsize=(5.5,5.5*0.7), constrained_layout=True)
gs = fig.add_gridspec(nrow, ncol,height_ratios=[1,1])   

for i in range(nrow):
# for pt in range(1):
    
    #======================================================================================================
    # PART 1. determine useful time period
    date_start_obj = datetime.datetime.strptime(date_starts[i], time_format)
    date_end_obj = datetime.datetime.strptime(date_ends[i], time_format)
    
    # determine gris' lat and long
    [xid,yid]=grids[i]
    lons = longitude[xid,yid]
    lats = latitude[xid,yid]   

    #======================================================================================================
    # PART 2. read historical nldas data
    time_nldas, pcp_nldas, tmean_nldas = read_nldas(nldas_dir,date_start_obj, date_end_obj, mask_xy)

    pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
    vmin_pcp = np.nanmin(pcp_nldas_mean)
    vmax_pcp = np.nanpercentile(pcp_nldas_mean,99)

    tmean_nldas_mean = np.nanmean(tmean_nldas, axis=0) #(y,x)
    vmin_tmean = np.nanmin(tmean_nldas_mean)
    vmax_tmean = np.nanmax(tmean_nldas_mean)

    #=====================================================
    # PART 3. plot 
    # reference: https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots    
    for j in range(ncol):
        if j==0:
            cmap,data,vmin,vmax=plt.cm.jet, pcp_nldas_mean, vmin_pcp, vmax_pcp #plt.cm.Blues
            cbar_label='(mm/day)'              
        else:
            cmap,data,vmin,vmax=plt.cm.jet, tmean_nldas_mean, vmin_tmean, vmax_tmean #plt.cm.jet
            cbar_label='($^\circ$C)'

        # plot Basemap
        ax = fig.add_subplot(gs[i,j])
        m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx) # plot Basemap 

        # plot data
        im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

        # set colorbar
        cbar = fig.colorbar(im,ax=ax,pad=-0.05,orientation="horizontal")  
        cbar.ax.tick_params(labelsize='xx-small',pad=0.01, length=2)             
        cbar.set_label(label=cbar_label, size='xx-small', rotation='horizontal', labelpad=-0.5)

        # set title
        alpha = chr(ord('a') + i*ncol + j)
        if j == 0:
            var = 'Precip'
        elif j == 1:
            var = 'Tmean'
        time_str = datetime.datetime.strftime(time_nldas[0],'%b. %Y')
        title_str = '(%s) %s, NLDAS %s'%(alpha,time_str,var)
        ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
        
        # plot grids
        m.scatter(lons, lats, marker = 'o', color='none', edgecolors='k', zorder=5, alpha=0.8)#magenta
        
# save
output_filename = '%s_plot_grids_locations.png'%(test_folder)
fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
plt.close(fig)


print('Done')


Read gridinfo mask
plot


/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_max' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_min' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_range' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
<ipython-input-8-086f4fd6c81f>:133: RuntimeWarning: Mean of empty slice
  pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
<ipython-input-8-086f4fd6c81f>:137: RuntimeWarning: Mean of empty slice
  tmean_nldas_mean = np.nanmean(t

Done


### 2. plot comparison ###

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime
import random
from tqdm import tqdm

def read_nldas(file_dir, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):
        nldas_file = 'NLDAS_'+str(yr)+'.nc'
        nldas_path = os.path.join(file_dir, nldas_file)
        f_nldas = xr.open_dataset(nldas_path)
        
        if yr == start_yr:
            pcp_concat = f_nldas['pcp'].values[:] # (time, y, x). unit: mm/day
            tmean_concat = f_nldas['t_mean'].values[:] # (time, y, x). unit: degC
#             t_range_concat = f_nldas['t_range'].values[:]
            time_concat = f_nldas['time'].values[:]
        else:
            pcp_concat = np.concatenate((pcp_concat, f_nldas['pcp'].values[:]), axis = 0)
            tmean_concat = np.concatenate((tmean_concat, f_nldas['t_mean'].values[:]), axis = 0)
#             trange_concat = np.concatenate((trange_concat, f_nldas['t_range'].values[:]), axis = 0)
            time_concat = np.concatenate((time_concat, f_nldas['time'].values[:]), axis = 0)

    # get time mask from nldas data
    time_obj = pd.to_datetime(time_concat)    
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 

    # convert masked values to nan
    time_obj = time_obj[mask_t]
    pcp_concat=np.where(mask_xy==0,np.nan,pcp_concat[mask_t,:,:])
    tmean_concat=np.where(mask_xy==0,np.nan,tmean_concat[mask_t,:,:])

    return time_obj, pcp_concat, tmean_concat#, t_range_concat

def read_ens_mb(file_dir, mb, date_start_obj, date_end_obj, mask_xy):
    start_yr = int(datetime.datetime.strftime(date_start_obj,'%Y'))
    end_yr = int(datetime.datetime.strftime(date_end_obj,'%Y'))
    
    for yr in range(start_yr, end_yr+1):                
        filename='ens_forc.%d.%03d.nc'%(yr,mb)
        file = os.path.join(file_dir, filename)
        f=xr.open_dataset(file)
        time = f['time'].values[:]
        pcp = f['pcp'].values[:]
        tmean = f['t_mean'].values[:]
#        trange = f['t_range'].values[:]
        
        if yr == start_yr:
            time_concat = time
            pcp_concat = pcp
            tmean_concat = tmean
#             trange_concat = trange
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_concat = np.concatenate((pcp_concat, pcp), axis=0) # (time,y,x)
            tmean_concat = np.concatenate((tmean_concat, tmean), axis=0)
#             trange_concat = np.concatenate((trange_concat, trange), axis=0)            
    
    time_obj = pd.to_datetime(time_concat)   
    mask_t  = (time_obj >= date_start_obj) & (time_obj <= date_end_obj) 

    # convert masked values to nan
    time_obj = time_obj[mask_t]
    pcp_concat=np.where(mask_xy==0,np.nan,pcp_concat[mask_t,:,:])
    tmean_concat=np.where(mask_xy==0,np.nan,tmean_concat[mask_t,:,:])
    
    return time_obj, pcp_concat, tmean_concat#, trange_concat

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)
    return

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
test_folder =test_folders[-1]

regress_subforlder = 'gmet_regr'
ens_subforlder = 'gmet_ens_bc'

ens_num = 100 #100
time_format = '%Y-%m-%d'

dpi_value = 150
date_starts = ['2016-02-01','2016-08-01'] # list of start dates of plot time periods
date_ends = ['2016-02-29','2016-08-31']   # list of end dates of plot time periods
grids = [[40,265],[40,265]]               # xid and yid of the spatial distribution
# date_starts = ['2016-08-09']
# date_ends = ['2016-08-20']

random.seed(10)

output_dir=os.path.join(root_dir, 'scripts/step35_statistical_perturb_NLDAS')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
#======================================================================================================
# Read gridinfo mask
print('Read gridinfo mask')
# get xy mask from gridinfo.nc
f_gridinfo = xr.open_dataset(gridinfo_file)
mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
latitude = f_gridinfo['latitude'].values[:]
longitude = f_gridinfo['longitude'].values[:]

# prepare for basemap plot
llcrnrlon = longitude[0,0]
urcrnrlon = longitude[-1,-1]
llcrnrlat = latitude[0,0]
urcrnrlat = latitude[-1,-1]
lat_0=0.5*(llcrnrlat+urcrnrlat)
lon_0=0.5*(llcrnrlon+urcrnrlon)
(ny,nx)=np.shape(longitude)

# #======================================================================================================    
# # create a white-jet linear colormap
# print('Create colormap')
# # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
# cmap = mpl.cm.get_cmap('jet') # get the blue color of jet 
# c0 = cmap(0.1)
# # top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])
# top = mpl.colors.LinearSegmentedColormap.from_list("", ["white","white"])

# # combine two liner colormaps to create a
# # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
# bottom = mpl.cm.get_cmap('jet')
# newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.1))),bottom(np.linspace(0, 1, int(256*0.9)))))
# newcmp = mpl.colors.LinearSegmentedColormap.from_list("WhiteJet", newcolors)

#======================================================================================================
print('Read data and plot')
for pt in range(len(date_starts)):
# for pt in range(1):
    
    #======================================================================================================
    # PART 1. determine useful time period
    date_start_obj = datetime.datetime.strptime(date_starts[pt], time_format)
    date_end_obj = datetime.datetime.strptime(date_ends[pt], time_format)
    [xid,yid]=grids[pt]
    lons = longitude[xid,yid]
    lats = latitude[xid,yid]   

    #======================================================================================================
    # PART 2. read historical nldas data
    time_nldas, pcp_nldas, tmean_nldas = read_nldas(nldas_dir,date_start_obj, date_end_obj, mask_xy)

    #======================================================================================================
    # PART 3. read historical ensemble data
    print('Read ensemble member data')
    file_dir = os.path.join(result_dir,test_folder,ens_subforlder)

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        time_mb, pcp_mb, tmean_mb = read_ens_mb(file_dir, mb+1, date_start_obj, date_end_obj, mask_xy)
        (nt,nx,ny) = np.shape(pcp_mb) 
        pcp_mb,tmean_mb = np.reshape(pcp_mb,(nt,nx,ny,1)),np.reshape(tmean_mb,(nt,nx,ny,1))

        if mb == 0:
            time_gmet_ens,pcp_gmet_ens,tmean_gmet_ens = time_mb,pcp_mb,tmean_mb
        else:
            pcp_gmet_ens = np.concatenate((pcp_gmet_ens,pcp_mb), axis=-1)
            tmean_gmet_ens = np.concatenate((tmean_gmet_ens,tmean_mb), axis=-1)
        pbar.update(1)
    pbar.close()
    del time_mb, pcp_mb, tmean_mb

    #=====================================================
    # PART 4. PRECIPITATION
    #=====================================================
    #=====================================================
    # 1.  perturb NLADS Precip
    print('perturb NLADS Precip')
    # --- assume Precip ~ Gamma ---
    (nt,nx,ny)=np.shape(pcp_nldas)
    pcp_stat_ens = np.empty((nt,nx,ny,ens_num))
    pcp_stat_ens[:] = np.NaN

    cp2 = 0.25**2
    shape = 1/cp2 * np.ones_like(pcp_nldas)
    scale = cp2 * pcp_nldas

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        pcp_stat_ens[...,mb]=np.random.gamma(shape, scale)
        pbar.update(1)
    pbar.close()

    #=====================================================
    # 2. calculate Precip std and mean for plot
    pcp_nldas_mean = np.nanmean(pcp_nldas, axis=0) #(y,x)
    pcp_stat_std_mean = np.nanmean(np.nanstd(pcp_stat_ens, axis=-1), axis=0) 
    pcp_gmet_std_mean = np.nanmean(np.nanstd(pcp_gmet_ens, axis=-1), axis=0)
    pcp_std_diff = pcp_gmet_std_mean - pcp_stat_std_mean
                      
    #=====================================================
    # 3. calculate Precip plot/colorbar range
    vmin_pcp = np.nanmin(pcp_nldas_mean)
    vmax_pcp = np.nanpercentile(pcp_nldas_mean,99)
    
    vmin_pcp_std = np.nanmin([np.nanmin(pcp_stat_std_mean),np.nanmin(pcp_gmet_std_mean)])
    vmax_pcp_std = np.nanmax([np.nanpercentile(pcp_stat_std_mean,99.5), np.nanpercentile(pcp_gmet_std_mean,99.5)])

    lim=max([abs(np.nanmin(pcp_std_diff)),abs(np.nanmax(pcp_std_diff))])
    vmin_pcp_std_diff=(-1)*(lim)
    vmax_pcp_std_diff=lim

    #=====================================================
    # 4. plot Precip
    # reference: https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots
    print('plot Precip')
    nrow = 2 # Wet, dry events
    ncol = 3 # NLDAS, GMET std, statistical std    
    fig = plt.figure(figsize=(7.08,7.08*0.5), constrained_layout=True)
    gs = fig.add_gridspec(nrow, ncol,height_ratios=[1,0.8])   
    
    for i in range(nrow):               
        #=====================================================
        if i == 0: # spatial plot (basemap)
            for j in range(ncol):
                if j==0:
                    cmap,vmin,vmax=plt.cm.jet, vmin_pcp, vmax_pcp #plt.cm.Blues
                    data,title_str=pcp_nldas_mean,'(a) NLDAS Precip'               
                else:
                    cmap,vmin,vmax=plt.cm.jet, vmin_pcp_std, vmax_pcp_std #plt.cm.jet
                    if j == 1:
                        data,title_str=pcp_gmet_std_mean,'(b) Ensemble std. dev.'
                    elif j==2:
                        data,title_str=pcp_stat_std_mean,'(c) Non-regression std. dev.'

                # plot Basemap
                ax = fig.add_subplot(gs[i,j])
                m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx) # plot Basemap 

                # plot data
                im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

                # set colorbar
                cbar = fig.colorbar(im,ax=ax,pad=0.005,orientation="horizontal")  
                cbar.ax.tick_params(labelsize='xx-small',pad=0.01, length=2)             
                cbar.set_label(label='(mm/day)', size='xx-small', rotation='horizontal', labelpad=-0.5)

                # set title
                ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
                
                # plot grids location
                if j == 0:
                    m.scatter(lons, lats, marker = 'o', color='none', 
                              edgecolors='red', zorder=5, alpha=0.8)#magenta
                
        #=====================================================
        elif i == 1: # temporal plot (boxplot)
            ax = fig.add_subplot(gs[i,:])

            # extract point data
            nldas_point = pcp_nldas[:,xid,yid]            
            data_left = pcp_gmet_ens[:,xid,yid,:]
            data_right = pcp_stat_ens[:,xid,yid,:]  

            data_left = data_left.tolist()
            data_right = data_right.tolist()
            
            # boxplot left data
            positions=np.array(range(len(data_left)))*2.0-0.4
            color='blue'
            flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                          linestyle='none', markeredgecolor=color)
            bpl = ax.boxplot(data_left, positions=positions, widths=0.6, 
                             flierprops=flierprops, medianprops=dict(alpha=.8))
            set_box_color(bpl, color) # colors are from http://colorbrewer2.org/
            sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1,zorder=2)

            # boxplot right data
            positions=np.array(range(len(data_left)))*2.0+0.4
            color='red'
            flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                          linestyle='none', markeredgecolor=color)
            bpr = ax.boxplot(data_right, positions=positions, widths=0.6, 
                             flierprops=flierprops, medianprops=dict(alpha=.8))
            set_box_color(bpr, color)
            sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1,zorder=2)

            # Add a horizontal grid to the plot, but make it very light in color
            # so we can use it for reading data values but not be distracting
            ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
            ax.set_axisbelow(True)
            
            # xticks
            ticks = [datetime.datetime.strftime(t,'%d') for t in time_nldas]
            ax.set_xticklabels(ticks)
            ax.set_xticks(np.array(range(len(data_left)))*2.0)
            ax.tick_params(axis='both',labelsize='xx-small')

            # draw temporary red and blue lines and use them to create a legend
            ax.legend([sc, bpl["boxes"][0], bpr["boxes"][0]], ['NLADS', 'Ensemble', 'Non-regression'], 
                      loc='upper right',fontsize='xx-small')

            ax.set_xlabel('Date', fontsize='xx-small') 
            ax.set_ylabel('Precip (mm/day)', fontsize='xx-small') 

            # set title
            title_str= '(d) Precip in ' + datetime.datetime.strftime(time_nldas[0],'%B %Y')                         
            ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
    
    # save
    output_filename = '%s_Precip_%s_xid%d_yid%d.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)

    #=====================================================
    # PART 5. Tmean
    #=====================================================
    #======================================================================================================
    # 1.  perturb NLADS Tmean
    print('perturb NLADS Tmean')
    # ---assume mean Temp ~ Normal ---
    (nt,nx,ny)=np.shape(tmean_nldas)
    tmean_stat_ens = np.empty((nt,nx,ny,ens_num))
    tmean_stat_ens[:] = np.NaN

    mean = tmean_nldas
    std = 0.25 # degree

    pbar = tqdm(total=ens_num)
    for mb in range(ens_num):
        tmean_stat_ens[...,mb]=np.random.normal(mean, std)
        pbar.update(1)
    pbar.close()

    #=====================================================
    # 2. calculate Tmean std and mean for plot
    tmean_nldas_mean = np.nanmean(tmean_nldas, axis=0) #(y,x)
    tmean_stat_std_mean = np.nanmean(np.nanstd(tmean_stat_ens, axis=-1), axis=0)                      
    tmean_gmet_std_mean = np.nanmean(np.nanstd(tmean_gmet_ens, axis=-1), axis=0)
    tmean_std_diff = tmean_gmet_std_mean - tmean_stat_std_mean
                       
    #=====================================================
    # 3. calculate Tmean plot/colorbar range
    vmin_tmean = np.nanmin(tmean_nldas_mean)
    vmax_tmean = np.nanmax(tmean_nldas_mean)
    
    vmin_tmean_std = np.nanmin([np.nanmin(tmean_stat_std_mean),np.nanmin(tmean_gmet_std_mean)])
    vmax_tmean_std = np.nanmax([np.nanpercentile(tmean_stat_std_mean,99.5),np.nanpercentile(tmean_gmet_std_mean,99.5)])

    #=====================================================
    # 4. plot Tmean
    # reference: https://stackoverflow.com/questions/16592222/matplotlib-group-boxplots
    print('plot')
    nrow = 2 # Wet, dry events
    ncol = 3 # NLDAS, GMET std, statistical std    
    fig = plt.figure(figsize=(7.08,7.08*0.5), constrained_layout=True)
    gs = fig.add_gridspec(nrow, ncol,height_ratios=[1,0.8])   
    
    for i in range(nrow):               
        #=====================================================
        if i == 0: # spatial plot (basemap)
            for j in range(ncol):
                if j==0:
                    cmap,vmin,vmax=plt.cm.jet, vmin_tmean, vmax_tmean #plt.cm.Blues
                    data,title_str=tmean_nldas_mean,'(a) NLDAS Tmean'               
                else:
                    cmap,vmin,vmax=plt.cm.jet, vmin_tmean_std, vmax_tmean_std #plt.cm.jet
                    if j == 1:
                        data,title_str=tmean_gmet_std_mean,'(b) Ensemble std. dev.'
                    elif j==2:
                        data,title_str=tmean_stat_std_mean,'(c) Non-regression std. dev.'

                # plot Basemap
                ax = fig.add_subplot(gs[i,j])
                m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx) # plot Basemap 

                # plot data
                im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)

                # set colorbar
                cbar = fig.colorbar(im,ax=ax,pad=0.005,orientation="horizontal")  
                cbar.ax.tick_params(labelsize='xx-small',pad=0.01, length=2)             
                cbar.set_label(label='($^\circ$C)', size='xx-small', rotation='horizontal', labelpad=-0.5)

                # set title
                ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
                
                # plot grids location
                if j == 0:
                    m.scatter(lons, lats, marker = 'o', color='none', 
                              edgecolors='red', zorder=5, alpha=0.8)#magenta
                    
        #=====================================================
        elif i == 1: # temporal plot (boxplot)
            ax = fig.add_subplot(gs[i,:])

            # extract point data
            nldas_point = tmean_nldas[:,xid,yid]            
            data_left = tmean_gmet_ens[:,xid,yid,:]
            data_right = tmean_stat_ens[:,xid,yid,:]  

            data_left = data_left.tolist()
            data_right = data_right.tolist()
            
            # boxplot left data
            positions=np.array(range(len(data_left)))*2.0-0.4
            color='blue'
            flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                          linestyle='none', markeredgecolor=color)
            bpl = ax.boxplot(data_left, positions=positions, widths=0.6, 
                             flierprops=flierprops, medianprops=dict(alpha=.8))
            set_box_color(bpl, color) # colors are from http://colorbrewer2.org/
            sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1)

            # boxplot right data
            positions=np.array(range(len(data_left)))*2.0+0.4
            color='red'
            flierprops = dict(marker='+', markerfacecolor='none', markersize=2,
                          linestyle='none', markeredgecolor=color)
            bpr = ax.boxplot(data_right, positions=positions, widths=0.6, 
                             flierprops=flierprops, medianprops=dict(alpha=.8))
            set_box_color(bpr, color)
            sc = ax.scatter(positions,nldas_point,s=4, c='k',alpha=1)

            # Add a horizontal grid to the plot, but make it very light in color
            # so we can use it for reading data values but not be distracting
            ax.yaxis.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
            ax.set_axisbelow(True)
            
            # xticks
            ticks = [datetime.datetime.strftime(t,'%d') for t in time_nldas]
            ax.set_xticklabels(ticks)
            ax.set_xticks(np.array(range(len(data_left)))*2.0)
            ax.tick_params(axis='both',labelsize='xx-small')

            # draw temporary red and blue lines and use them to create a legend
            ax.legend([sc, bpl["boxes"][0], bpr["boxes"][0]], ['NLADS', 'Ensemble', 'Non-regression'], 
                      loc='upper right',fontsize='xx-small')

            ax.set_xlabel('Date', fontsize='xx-small') 
            ax.set_ylabel('Tmean ($^\circ$C)', fontsize='xx-small') 

            # set title
            title_str= '(d) Tmean in ' + datetime.datetime.strftime(time_nldas[0],'%B %Y')                         
            ax.set_title(title_str, fontsize='xx-small', fontweight='semibold')
    
    # save
    output_filename = '%s_Tmean_%s_xid%d_yid%d.png'%(test_folder, datetime.datetime.strftime(time_nldas[0],'%B_%Y'),xid, yid)
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)
print('Done')


Read gridinfo mask
Read data and plot


/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_max' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_min' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/lib/python3.8/site-packages/xarray/conventions.py:487: SerializationWarning: variable 't_range' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
  0%|          | 0/100 [00:00<?, ?it/s]

Read ensemble member data


 95%|█████████▌| 95/100 [03:31<00:14,  2.99s/it]

In [11]:
np.where(pcp_nldas_mean>42),pcp_nldas_mean[pcp_nldas_mean>42]

<ipython-input-11-474b526982e1>:1: RuntimeWarning: invalid value encountered in greater
  np.where(pcp_nldas_mean>42),pcp_nldas_mean[pcp_nldas_mean>42]


((array([38, 40]), array([261, 265])),
 array([42.10705, 42.34092], dtype=float32))

### BACK UP ###